# 2.10 Dual basis functions


### Canonical interpolation

In [1]:
import netgen.gui
from ngsolve import *
from netgen.geom2d import unit_square

mesh = Mesh(unit_square.GenerateMesh(maxh=2))

func = x*x*x*x

fes = H1(mesh, order=3, low_order_space=False)
u,v = fes.TnT()
vdual = v.Operator("dual")

a = BilinearForm(fes)
a += u*vdual*dx + u*vdual*dx(element_vb=BND) + u*vdual*dx(element_vb=BBND)
a.Assemble()
f = LinearForm(fes)
f += func*vdual*dx + func*vdual*dx(element_vb=BND) + func*vdual*dx(element_vb=BBND)
f.Assemble()

# print (a.mat)
# print (f.vec)

gfu = GridFunction(fes)
# not exatly the vertex values ...
gfu.Set(func)
print ("gfu with Oswald interpolation:", gfu.vec)


# interpolation in vertices
gfu.vec.data = a.mat.Inverse() * f.vec
print ("gfu with canonical interpolaton:", gfu.vec)
Draw (gfu)

gfu with Oswald interpolation:  -0.0223792
 0.991843
 0.977621
 -0.00815655
 3.34984
 2.19912
 3.34984
       2
 0.00660131
 4.29197e-15
 0.00660131
 -2.1896e-14
 3.34984
 -1.80088
 -2.17601
 1.82399


gfu with canonical interpolaton:        0
       1
       1
       0
     3.6
       2
     3.6
       2
       0
       0
 3.33067e-15
 -3.33067e-15
     3.6
      -2
      -2
       2




In [2]:
import netgen.gui
from ngsolve import *
from netgen.geom2d import unit_square

mesh = Mesh(unit_square.GenerateMesh(maxh=2))

fesh1 = VectorH1(mesh, order=2)
feshdiv = HDiv(mesh, order=2)

gfuh1 = GridFunction(fesh1)
gfuh1.Set ( (x*x,y*y) )

amixed = BilinearForm(trialspace=fesh1, testspace=feshdiv)
ahdiv = BilinearForm(feshdiv)

u,v = feshdiv.TnT()
vdual = v.Operator("dual")
uh1 = fesh1.TrialFunction()

n = specialcf.normal(mesh.dim)
def tang(v) : return v - (v*n)*n

dS = dx(element_boundary=True)
ahdiv += u*vdual * dx
ahdiv += u*vdual * dS
ahdiv.Assemble()

amixed += uh1*vdual * dx
amixed += uh1*vdual * dS
amixed.Assemble()

print ("ahdiv =", ahdiv.mat)
print ("amixed =", amixed.mat)


gfuhdiv = GridFunction(feshdiv, name="ucomp")
transform = ahdiv.mat.Inverse() @ amixed.mat
gfuhdiv.vec.data = transform * gfuh1.vec

Draw (gfuh1)
Draw (gfuhdiv, mesh, "uhdiv")

# input ("key")

eblocks = []
for e in mesh.edges:
    eblocks.append ( feshdiv.GetDofNrs(e) )
fblocks = []
for f in mesh.faces:
    fblocks.append ( feshdiv.GetDofNrs(f) )

print (eblocks)
print (fblocks)

einv = ahdiv.mat.CreateBlockSmoother(eblocks)
finv = ahdiv.mat.CreateBlockSmoother(fblocks)

transform = (einv + finv @ (IdentityMatrix() - ahdiv.mat @ einv)) @ amixed.mat
gfuhdiv.vec.data = transform * gfuh1.vec

# input ("key")

temp = amixed.mat.CreateColVector()
temp.data = amixed.mat * gfuh1.vec
gfuhdiv.vec[:] = 0
einv.Smooth (gfuhdiv.vec, temp)
finv.Smooth (gfuhdiv.vec, temp)


ahdiv = Row 0:   0: -1   1: 0   3: 0   5: 0   6: -1.9984e-15   7: 0   8: 0   11: 0   12: 0   15: 0   16: 0   17: 0
Row 1:   0: 0   1: -2.82843   2: 0   3: 0   4: 0   5: 0   6: 0   7: 0   8: -5.66387e-15   9: 0   10: 0   11: 0   12: 0   13: 0   14: 0   15: 0   16: 0   17: 0   18: 0   19: 0   20: 0
Row 2:   1: 0   2: -1   4: 0   7: 0   8: 0   9: 0   10: -2.01228e-15   13: 0   14: 0   18: 0   19: 0   20: 0
Row 3:   0: 0   1: 0   3: -1   5: 0   6: 0   7: 0   8: 0   11: 0   12: -1.9984e-15   15: 0   16: 0   17: 0
Row 4:   1: 0   2: 0   4: -1   7: 0   8: 0   9: 0   10: 0   13: 0   14: -1.9984e-15   18: 0   19: 0   20: 0
Row 5:   0: 1.38778e-17   1: 0   3: 0   5: 0.166667   6: 0   7: 0   8: 0   11: 0   12: 0   15: 0   16: 0   17: 0
Row 6:   0: 4.05478e-15   1: 0   3: 0   5: 0   6: 0.1   7: 0   8: 0   11: 0   12: 0   15: 0   16: 0   17: 0
Row 7:   0: 0   1: 1.38778e-17   2: 0   3: 0   4: 0   5: 0   6: 0   7: 0.471405   8: 1.38778e-17   9: 0   10: 0   11: 0   12: 0   13: 0   14: 0   15: 0   16:

## Auxiliary Space Preconditioning
use H1 preconditioner for what ? MCS ? 